In [5]:
import pandas as pd

import os

import numpy as np

import tensorflow as tf
from tensorflow import keras

In [3]:
df_comments = pd.read_csv('data/get_comment_threads_output/df_comment_thread.csv')

In [4]:
df_comments

,video_id,comment_thread_id,top_level_comment_id,author_display_name,author_channel_id,text_original
0,WyUyJKm-MiY,UgxmeuEGKaTnjr18AVZ4AaABAg,UgxmeuEGKaTnjr18AVZ4AaABAg,Imaan Jaffri,UCX6Ub2LzGnU6qjXVDS47EdQ,After listening to that podcast... Nathan dodg...
1,WyUyJKm-MiY,UgzlwPwyC3XlmzgzBr54AaABAg,UgzlwPwyC3XlmzgzBr54AaABAg,Karlee Nash,UCB9XxMYxVDoeLLVzf8M5M8A,Jeffree:Eyebrows never heard of her😂
2,WyUyJKm-MiY,UgynycPfXmKJePLzAEp4AaABAg,UgynycPfXmKJePLzAEp4AaABAg,Bernice Ayer,UC_FRgaHgKVLno4V_1iA3rlA,Hi bae
3,WyUyJKm-MiY,UgwamwomyWMY8bpiPjp4AaABAg,UgwamwomyWMY8bpiPjp4AaABAg,Molly Seymour,UCAn0o15lbsbLudv6J3hO3Kw,Jeffree: WTF I HAVE SOO Many subs!\nMe: i have...
4,WyUyJKm-MiY,Ugx7NWDtnzTkeACe5yJ4AaABAg,Ugx7NWDtnzTkeACe5yJ4AaABAg,Charbonelle Weese,UC4Qnz1VN47C4rqXSYT7Nj8g,Love the wig! Looking beautiful as always 😍
...,...,...,...,...,...,...
2056377,lOIFYf3sSnw,Ugw50lN281elkmpu2014AaABAg,Ugw50lN281elkmpu2014AaABAg,Lovely .A,UCQlbYd6E5ttfF60VOQrh60g,I have a pimple in the same place I feel speci...
2056378,lOIFYf3sSnw,Ugx0MzMiJfba86UFRBp4AaABAg,Ugx0MzMiJfba86UFRBp4AaABAg,Diana Thompson,UCRw9Um0-V_84eBgt56hVJfg,"Wooiiiii now listen Jackie , there was no need..."
2056379,lOIFYf3sSnw,UgzRG-y0DoBkLtpLl1J4AaABAg,UgzRG-y0DoBkLtpLl1J4AaABAg,AA W,UCn47Oo7c8MRtrlUXbAW4JYA,"After 2 kids, Jackie is bad for my bladder 😂😂😂..."
2056380,lOIFYf3sSnw,UgwgoR1qaPAv3NPtEih4AaABAg,UgwgoR1qaPAv3NPtEih4AaABAg,Brianna Taylor,UCeTugksWfd72bshf3HDBq8g,The talking pimple got you a foreverrrr subscr...
